## Performance of the CPL
Author: Clara Hoffmann \
Measures the performance of the CPL on the precise validation set.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import scipy as sp
from tqdm import tqdm
from scipy.integrate import simps, trapz
import matplotlib.pyplot as plt
import imageio
import multiprocessing
from helpers import (Fy, find_closest_element,  compute_coverage, #predict_single_density,
confidence_interval, confidence_interval, generate_fixed_terms, get_ci)
from density_predictor_class import density_predictor

In [2]:
# read in val data
B_zeta = np.load('../../../../data/commaai/extracted_coefficients/20201027_filtered_gaussian_resampled/Bzeta/B_zeta_val.npy')
B_zeta = B_zeta.reshape(B_zeta.shape[0], B_zeta.shape[2])
true_y = np.load('../../../../data/commaai/extracted_coefficients/20201027_filtered_gaussian_resampled/Bzeta/labels_val.npy')

density_path = '../../../../data/commaai/density/gaussian_density_filtered.csv'
density = pd.read_csv(density_path)

# number of points at which to evaluate the predictive densities
no_points = 500

In [3]:
dens = density_predictor(B_zeta, true_y, density, no_points)

In [5]:
result_va_ridge = dens.get_density('va_ridge')

  1%|          | 87/10472 [00:00<00:11, 866.59it/s]

computing densities for each observation


  6%|▋         | 676/10472 [00:00<00:01, 6758.40it/s]

computing mean prediction for each observation


 15%|█▌        | 1610/10472 [00:00<00:00, 16099.75it/s]

computing variance prediction for each observation


100%|██████████| 10472/10472 [00:00<00:00, 16315.23it/s]


In [6]:
result_hmc_ridge = dens.get_density('hmc_ridge')

100%|██████████| 10472/10472 [00:00<00:00, 16690.20it/s]


In [7]:
result_va_horse = dens.get_density('va_horseshoe')

100%|██████████| 10472/10472 [00:00<00:00, 16963.36it/s]


In [4]:
result_hmc_horse = dens.get_density('hmc_horseshoe')

100%|██████████| 10472/10472 [00:00<00:00, 16819.01it/s]


In [ ]:
result_va_ridge = dens.get_density('va_ridge')
result_hmc_ridge = dens.get_density('hmc_ridge')
result_va_horse = dens.get_density('va_horseshoe')
result_hmc_horse = dens.get_density('hmc_horseshoe')

In [12]:
print('Validation Performance: ' +
      '\nMSE:' +
      '\n CPL Benchmark: ' + str(np.mean(np.abs(true_y - 0)**2)) +
     '\n CPL VA/Ridge: ' + str(np.mean((true_y - np.array(result_va_ridge['mean predictions']))**2)) +
     '\n CPL HMC/Ridge: ' + str(np.mean((true_y - np.array(result_hmc_ridge['mean prediction']))**2)) +
     '\n CPL VA/Horseshoe: ' + str(np.mean((true_y - np.array(result_va_horse['mean prediction']))**2)) +
     '\n CPL HMC/Horseshoe: ' + str(np.mean((true_y - np.array(result_hmc_horse['mean prediction']))**2)) +
      
    '\nMAE:' +
      '\n CPL Benchmark: ' + str(np.mean(np.abs(true_y - 0))) +
     '\n CPL VA/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_va_ridge['mean predictions'])))) +
     '\n CPL HMC/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_ridge['mean prediction'])))) +
     '\n CPL VA/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_va_horse['mean prediction'])))) +
     '\n CPL HMC/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_horse['mean prediction'])))) +
      
      '\nAccuracy:' +
     '\n CPL Benchmark: ' + str(np.mean(np.abs(true_y - 0) < 6)) +
     '\n CPL VA/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_va_ridge['mean predictions'])) < 6)) +
     '\n CPL HMC/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_ridge['mean prediction'])) < 6)) +
     '\n CPL VA/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_va_horse['mean prediction'])) < 6)) +
     '\n CPL HMC/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_horse['mean prediction'])) < 6))
     )

Validation Performance: 
MSE:
 CPL Benchmark: 14.610926279602749
 CPL VA/Ridge: 8.742672781658126
 CPL HMC/Ridge: 8.738334359535159
 CPL VA/Horseshoe: 8.675970189106746
 CPL HMC/Horseshoe: 8.647669965997942
MAE:
 CPL Benchmark: 2.1570091673032854
 CPL VA/Ridge: 1.8209580931671063
 CPL HMC/Ridge: 1.8200717288647659
 CPL VA/Horseshoe: 1.729607107517588
 CPL HMC/Horseshoe: 1.7259873478202725
Accuracy:
 CPL Benchmark: 0.9281894576012223
 CPL VA/Ridge: 0.9805194805194806
 CPL HMC/Ridge: 0.9806149732620321
 CPL VA/Horseshoe: 0.9764132925897632
 CPL HMC/Horseshoe: 0.9764132925897632


In [ ]:
df = pd.DataFrame({'va_ridge' : result_va_ridge['densities'],
                 'hmc_ridge' : result_hmc_ridge['densities'],
                 'va_horse' : result_va_horse['densities'],
                 'hmc_horse' : result_hmc_horse['densities']})

In [ ]:
df.to_csv('../../../../data/commaai/predictions/cpl/densities.csv')